<a href="https://colab.research.google.com/github/libanos123/Backtesting/blob/main/Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
#pip install ccxt -q

In [34]:
import ccxt
import pandas as pd
from Strategy_Bank import SMA_Strategy


SYMBOLS = ["BTC/USDT:USDT", "ETH/USDT:USDT", "SOL/USDT:USDT", "BNB/USDT:USDT"]
TIMEFRAME = "15m"
LIMIT = 500
FEE = 0.0006   # Bitget futures taker fee (approx)
#-------FETCH DATA--------
def fetch_ohlcv(exchange, symbol):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=TIMEFRAME, limit=LIMIT)
    return pd.DataFrame(
        ohlcv,
        columns=["timestamp", "open", "high", "low", "close", "volume"]
    )

basic backtester

In [35]:
#‐------BACKTEST ENGINE-----

def backtest(df, strategy):
    position = 0
    entry_price = 0
    pnl = 0
    trades = 0
    wins = 0

    for i in range(len(df)):
        data_slice = df.iloc[: i + 1]
        signal = strategy.generate_signal(data_slice)
        price = df["close"].iloc[i]

        # OPEN
        if position == 0:
            if signal == "BUY":
                position = 1
                entry_price = price
                trades += 1

            elif signal == "SELL":
                position = -1
                entry_price = price
                trades += 1

        # CLOSE
        elif position == 1 and signal == "SELL":
            trade_pnl = (price - entry_price) / entry_price - FEE
            pnl += trade_pnl
            wins += trade_pnl > 0
            position = 0

        elif position == -1 and signal == "BUY":
            trade_pnl = (entry_price - price) / entry_price - FEE
            pnl += trade_pnl
            wins += trade_pnl > 0
            position = 0


    winrate = (wins / trades) * 100 if trades > 0 else 0

    return {
        "Trades": trades,
        "Winrate (%)": round(winrate, 2),
        "Total PnL (%)": round(pnl * 100, 2),
    }


#------SCREENER--------
def run_screener():
    exchange = ccxt.bitget({
        "options": {
            "defaultType": "swap"
        }
    })

    results = []

    for symbol in SYMBOLS:
        try:
            df = fetch_ohlcv(exchange, symbol)
            strategy = SMA_Strategy()   # 👈 NO arguments
            stats = backtest(df, strategy)
            stats["Symbol"] = symbol
            results.append(stats)

        except Exception as e:
            print(f"Error with {symbol}: {e}")

    if not results:
        print("No results generated.")
        return

    table = pd.DataFrame(results).set_index("Symbol")
    print("\n===== BOT SCREENER RESULTS =====\n")
    print(table)

    return table


if __name__ == "__main__":
    run_screener()


===== BOT SCREENER RESULTS =====

               Trades  Winrate (%)  Total PnL (%)
Symbol                                           
BTC/USDT:USDT      86        23.26          -6.13
ETH/USDT:USDT      81        29.63          -0.98
SOL/USDT:USDT      79        22.78          -7.84
BNB/USDT:USDT      91        21.98         -14.94
